In [ ]:
%pip install pyhive

In [ ]:
from pyhive import hive


# Connect to Hive Metastore
conn = hive.Connection(host='10.11.0.10', port=10000, username='anonymous')

cursor = conn.cursor()
cursor.execute('SHOW DATABASES')

print("Databases:")
for database in cursor.fetchall():
    print(database)

In [ ]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .appName("GCS_to_Minio_Transform")
         # Enable Hive support so we can integrate with Hive metastore:
         .enableHiveSupport()
         # GCS configuration (assuming you've put the GCS connector jar in the classpath)
         .config("spark.hadoop.google.cloud.auth.service.account.enable", "true")
         .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/home/developer/keys/gcp-keys/advocado-billing/adhoc-billing-advocado-billing--5e634048a179.json")
         # Minio configuration
         .config("spark.hadoop.fs.s3a.endpoint", "https://minio.local-stl.n90.co:9000")
         .config("spark.hadoop.fs.s3a.access.key", "5flC58yYrT3qHV8OA0l7")
         .config("spark.hadoop.fs.s3a.secret.key", "Y0tDcY4evyUvSqX6NlAoWsD4Rkn2VIdb4uH2lGu2")
         .config("spark.hadoop.fs.s3a.path.style.access", "true")
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .getOrCreate())

In [ ]:
df = spark.read.parquet("s3://n90-data-lake-stl/veil/encodings_v2/")

In [ ]:
import pyarrow.dataset as ds
import pyarrow.fs as fs
import pandas as pd


# def load_encodings_for_detections(storage_options, bucket_name, hive_table_path, platform='s3', scheme='https', region='bws-stl'):
    # Construct the S3 URI
scheme='https'
region='bws-stl'
bucket_name='n90-data-lake-stl'
hive_table_path='veil/encodings_v2'
access_key='5flC58yYrT3qHV8OA0l7'
secret_key='Y0tDcY4evyUvSqX6NlAoWsD4Rkn2VIdb4uH2lGu2'
endpoint_override='https://s4.api-access.com:9000'
    
s3_uri = f"{bucket_name}/{hive_table_path}/"
print(s3_uri)

# Initialize the S3FileSystem with MinIO configuration
# access_key = storage_options.get('key')
print(access_key)
# secret_key = storage_options.get('secret')
print(secret_key)
# endpoint_override = storage_options.get('client_kwargs').get('endpoint_url')
print(endpoint_override)
s3_fs = fs.S3FileSystem(
    access_key=access_key,
    secret_key=secret_key,
    endpoint_override=endpoint_override,
    scheme=scheme,  # Use 'https' if SSL is enabled
    region=region  # MinIO requires a region; can be arbitrary
)

# Define the partitioning scheme (Hive-style)
partitioning = ds.partitioning(
    flavor='hive'
)

# Create the dataset
dataset = ds.dataset(
    s3_uri,
    format='parquet',
    partitioning=partitioning,
    filesystem=s3_fs
)


# Convert the dataset to a PyArrow Table
table = dataset.to_table()
table_subset = table.select(['encoding_id', 'isci', 'aeis_id', 'encoded_timestamp', 'format_id', 'format_name', 'customer_id', 'customer_name', 'sfdc_account_id', 'sfdc_account_name', 'sfdc_advertiser_id','attributes_cable_estimate', 'attributes_spot_estimate',
                            'encoder_group_id',  'encoder_id', 'encoder_group_name', 'length_in_seconds',
                            'billing_last_updated', 'billing_last_audit_id', 'clone_of'])

# Optionally, convert to pandas DataFrame
df = table_subset.to_pandas()

# return df

In [ ]:
import pyarrow.dataset as ds
import pyarrow.fs as fs
import pandas as pd

minio_endpoint = 'https://s4.api-access.com:9000'  # Replace with your MinIO endpoint
minio_access_key = '5flC58yYrT3qHV8OA0l7'
minio_secret_key = 'Y0tDcY4evyUvSqX6NlAoWsD4Rkn2VIdb4uH2lGu2'
bucket_name = 'n90-data-lake-stl'
hive_table_path = 'veil/encodings_v2/'  # Ensure it ends with '/'


def load_encodings_for_detections(storage_options, bucket_name, hive_table_path, platform='s3', scheme='https', region='bws-stl'):
    # Construct the S3 URI
    s3_uri = f"{bucket_name}/{hive_table_path}/"

    # Initialize the S3FileSystem with MinIO configuration
    s3_fs = fs.S3FileSystem(
        access_key=storage_options.get('access_key'),
        secret_key=storage_options.get('secret_key'),
        endpoint_override=storage_options.get('client_kwargs').get('endpoint_url'),
        scheme='https',  # Use 'https' if SSL is enabled
        region=region  # MinIO requires a region; can be arbitrary
    )

    # Define the partitioning scheme (Hive-style)
    partitioning = ds.partitioning(
        flavor='hive'
    )

    # Create the dataset
    dataset = ds.dataset(
        s3_uri,
        format='parquet',
        partitioning=partitioning,
        filesystem=s3_fs
    )

    # Convert the dataset to a PyArrow Table
    table = dataset.to_table()
    table_subset = table.select(['encoding_id', 'isci', 'aeis_id', 'encoded_timestamp', 'format_id', 'format_name', 'customer_id', 'customer_name', 'sfdc_account_id', 'sfdc_account_name', 'sfdc_advertiser_id','attributes_cable_estimate', 'attributes_spot_estimate',
                             'encoder_group_id',  'encoder_id', 'encoder_group_name', 'length_in_seconds',
                             'billing_last_updated', 'billing_last_audit_id', 'clone_of'])

    # Optionally, convert to pandas DataFrame
    df = table_subset.to_pandas()

    return df
# Construct the S3 URI
s3_uri = f"{bucket_name}/{hive_table_path}/"

# Initialize the S3FileSystem with MinIO configuration
s3_fs = fs.S3FileSystem(
    access_key=minio_access_key,
    secret_key=minio_secret_key,
    endpoint_override=minio_endpoint,
    scheme='https',  # Use 'https' if SSL is enabled
    region='bws-stl'  # MinIO requires a region; can be arbitrary
)

# Define the partitioning scheme (Hive-style)
partitioning = ds.partitioning(
    flavor='hive'
)

# Create the dataset
dataset = ds.dataset(
    s3_uri,
    format='parquet',
    partitioning=partitioning,
    filesystem=s3_fs
)

# Convert the dataset to a PyArrow Table
table = dataset.to_table()
table_subset = table.select(['encoding_id', 'isci', 'aeis_id', 'encoded_timestamp', 'format_id', 'format_name', 'customer_id', 'customer_name', 'sfdc_account_id', 'sfdc_account_name', 'sfdc_advertiser_id','attributes_cable_estimate', 'attributes_spot_estimate',
                             'encoder_group_id',  'encoder_id', 'encoder_group_name', 'length_in_seconds',
                             'billing_last_updated', 'billing_last_audit_id', 'clone_of'])

# Optionally, convert to pandas DataFrame
df = table_subset.to_pandas()

print(df.head())

In [ ]:
for col in df.columns:
    print(f'Col Name: {col}')
    print(f'Col Type: {df[col].dtype}')
    print(f'Col Sample Values: {df[col][1006510]}')

In [ ]:

import yaml
file_path='/home/developer/keys/project-keys/colab-settings.yaml'
try:
    with open(file_path, 'r') as file:
        config = yaml.safe_load(file)
except FileNotFoundError:
    raise FileNotFoundError(f"Configuration file not found at: {file_path}")
except yaml.YAMLError as e:
    raise ValueError(f"Error parsing YAML configuration file: {e}")

# minio_endpoint = 'https://s4.api-access.com:9000'  # Replace with your MinIO endpoint
# minio_access_key = '5flC58yYrT3qHV8OA0l7'
# minio_secret_key = 'Y0tDcY4evyUvSqX6NlAoWsD4Rkn2VIdb4uH2lGu2'
# bucket_name = 'n90-data-lake-stl'
# hive_table_path = 'veil/encodings_v2/'  # Ensure it ends with '/'

# resp = {}
# resp = core_functions.initialize_clients(service_account_secret_name='SA_ADHOC_BILLING')
# resp2 = core_functions.initialize_clients(service_account_secret_name='SA_N90_CORE_APPS')

# config = resp.get('config')
storage_options = config.get('VEIL_GCS_STORAGE_OPTIONS')
storage_options

In [ ]:

storage_options = config.get('VEIL_GCS_STORAGE_OPTIONS')
storage_options
bucket_name = config.get('veil_billing').get('billing_gcs_bucket_id')
hive_table_path = 'encodings_v2'
# bucket_name = 'n90-data-lake-stl'



# Construct the GCS URI
gcs_uri = f"{bucket_name}/{hive_table_path}/"

# Initialize the GcsFileSystem with GCS configuration
gcs_fs = fs.GcsFileSystem(
    access_token=storage_options.get('access_token')  # Provide GCS access token if needed
)

# Define the partitioning scheme (Hive-style)
partitioning = ds.partitioning(
    flavor='hive'
)

# Create the dataset
dataset = ds.dataset(
    gcs_uri,
    format='parquet',
    partitioning=partitioning,
    filesystem=gcs_fs
)

# Convert the dataset to a PyArrow Table
table = dataset.to_table()